## Capstone Project - The Battle of Neighborhoods 1

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20201111120553-0000
KERNEL_ID = 96f2c238-bf0a-4807-8f19-dbd9730a3033
Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.6.20          |   py37he5f6b98_2         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                   

zipp-3.4.0           | 11 KB     | ##################################### | 100% 
idna-2.10            | 52 KB     | ##################################### | 100% 
pysocks-1.7.1        | 27 KB     | ##################################### | 100% 
six-1.15.0           | 14 KB     | ##################################### | 100% 
cffi-1.14.3          | 223 KB    | ##################################### | 100% 
pyrsistent-0.17.3    | 89 KB     | ##################################### | 100% 
python-dateutil-2.8. | 220 KB    | ##################################### | 100% 
importlib-metadata-2 | 28 KB     | ##################################### | 100% 
jinja2-2.11.2        | 93 KB     | ##################################### | 100% 
pycparser-2.20       | 94 KB     | ##################################### | 100% 
pyopenssl-19.1.0     | 47 KB     | ##################################### | 100% 
altair-4.1.0         | 614 KB    | ##################################### | 100% 
libblas-3.9.0        | 11 KB

In [32]:
#foursquare credentials
CLIENT_ID = '33ASWGTBDUB5DPU1Q35V3UGOFCESBNQOQPJ23QT2QV2RTYG3'
CLIENT_SECRET = 'NIC4SFFHGVWIQ043S1BEKQRKAISNYODGGMDLJ4NSQ3THF103'
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33ASWGTBDUB5DPU1Q35V3UGOFCESBNQOQPJ23QT2QV2RTYG3
CLIENT_SECRET:NIC4SFFHGVWIQ043S1BEKQRKAISNYODGGMDLJ4NSQ3THF103


In [45]:
#milan position
address = 'Milano, Italy'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

45.4668 9.1905


In [58]:
#first search
search_query = 'pub'
radius = 3000

In [59]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=33ASWGTBDUB5DPU1Q35V3UGOFCESBNQOQPJ23QT2QV2RTYG3&client_secret=NIC4SFFHGVWIQ043S1BEKQRKAISNYODGGMDLJ4NSQ3THF103&ll=45.4668,9.1905&v=20180604&query=pub&radius=3000&limit=50'

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fabf44bc699df34c894e31c'},
 'response': {'venues': [{'id': '4dfbadf81f6e91394d34c09b',
    'name': 'Bukowsky Pub',
    'location': {'address': 'Via Santa Sofia 21',
     'lat': 45.456723593271406,
     'lng': 9.18942226527423,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.456723593271406,
       'lng': 9.18942226527423}],
     'distance': 1124,
     'postalCode': '20122',
     'cc': 'IT',
     'city': 'Milano',
     'state': 'Lombardia',
     'country': 'Italia',
     'formattedAddress': ['Via Santa Sofia 21',
      '20122 Milano Lombardia',
      'Italia']},
    'categories': [{'id': '4d4b7105d754a06374d81259',
      'name': 'Food',
      'pluralName': 'Food',
      'shortName': 'Food',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1605104715',
    'hasPerk': False},
   {'id': '4b058879f964a520dfc722e3',
    'name': 'Old Fox P

In [82]:
#gro
venues = results['response']['venues']


dataframe = json_normalize(venues)
dataframe.head()

/opt/ibm/conda/miniconda/lib/python/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4dfbadf81f6e91394d34c09b,Bukowsky Pub,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1605104715,False,Via Santa Sofia 21,45.456724,9.189422,"[{'label': 'display', 'lat': 45.45672359327140...",1124,20122,IT,Milano,Lombardia,Italia,"[Via Santa Sofia 21, 20122 Milano Lombardia, I...",NaN,NaN
1,4b058879f964a520dfc722e3,Old Fox Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1605104715,False,"Piazza Sant'Agostino,1",45.458828,9.170789,"[{'label': 'display', 'lat': 45.458828, 'lng':...",1776,20123,IT,Milano,Lombardia,Italia,"[Piazza Sant'Agostino,1, 20123 Milano Lombardi...",86453663,NaN
2,4b66e951f964a520de2f2be3,Pub 24,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1605104715,False,via Pietro Borsieri 24,45.488326,9.188938,"[{'label': 'display', 'lat': 45.48832614998453...",2399,20159,IT,Milano,Lombardia,Italia,"[via Pietro Borsieri 24, 20159 Milano Lombardi...",NaN,NaN
3,4b058877f964a52062c722e3,Matricola Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1605104715,False,Viale Romagna 43,45.475796,9.223411,"[{'label': 'display', 'lat': 45.47579606003361...",2757,20133,IT,Milano,Lombardia,Italia,"[Viale Romagna 43, 20133 Milano Lombardia, Ita...",NaN,NaN
4,4be1c98bae55a593ec8e5b62,Old Camillo's Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1605104715,False,Via Ascanio Sforza,45.449996,9.177554,"[{'label': 'display', 'lat': 45.44999642615075...",2126,NaN,IT,Milano,Lombardia,Italia,"[Via Ascanio Sforza, Milano Lombardia, Italia]",NaN,NaN


In [83]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Bukowsky Pub,Food,Via Santa Sofia 21,45.456724,9.189422,"[{'label': 'display', 'lat': 45.45672359327140...",1124,20122,IT,Milano,Lombardia,Italia,"[Via Santa Sofia 21, 20122 Milano Lombardia, I...",NaN,4dfbadf81f6e91394d34c09b
1,Old Fox Pub,Pub,"Piazza Sant'Agostino,1",45.458828,9.170789,"[{'label': 'display', 'lat': 45.458828, 'lng':...",1776,20123,IT,Milano,Lombardia,Italia,"[Piazza Sant'Agostino,1, 20123 Milano Lombardi...",NaN,4b058879f964a520dfc722e3
2,Pub 24,Pub,via Pietro Borsieri 24,45.488326,9.188938,"[{'label': 'display', 'lat': 45.48832614998453...",2399,20159,IT,Milano,Lombardia,Italia,"[via Pietro Borsieri 24, 20159 Milano Lombardi...",NaN,4b66e951f964a520de2f2be3
3,Matricola Pub,Pub,Viale Romagna 43,45.475796,9.223411,"[{'label': 'display', 'lat': 45.47579606003361...",2757,20133,IT,Milano,Lombardia,Italia,"[Viale Romagna 43, 20133 Milano Lombardia, Ita...",NaN,4b058877f964a52062c722e3
4,Old Camillo's Pub,Pub,Via Ascanio Sforza,45.449996,9.177554,"[{'label': 'display', 'lat': 45.44999642615075...",2126,NaN,IT,Milano,Lombardia,Italia,"[Via Ascanio Sforza, Milano Lombardia, Italia]",NaN,4be1c98bae55a593ec8e5b62
5,The English Football Pub,Sports Bar,Via Valpetrosa 5,45.461999,9.185520,"[{'label': 'display', 'lat': 45.46199942740473...",660,20123,IT,Milano,Lombardia,Italia,"[Via Valpetrosa 5, 20123 Milano Lombardia, Ita...",NaN,4b058878f964a52091c722e3
6,Alabama Pub,Pub,"Via Carlo Farini, 66",45.493975,9.184910,"[{'label': 'display', 'lat': 45.49397527250103...",3056,NaN,IT,Milano,Lombardia,Italia,"[Via Carlo Farini, 66, Milano Lombardia, Italia]",NaN,4cf2f4ebcc61a35d46c91d9e
7,JIU pub,Pub,NaN,45.490022,9.217857,"[{'label': 'display', 'lat': 45.49002166505541...",3352,NaN,IT,NaN,NaN,Italia,[Italia],NaN,4d4efa56deabf04d01fcc724
8,pils pub,Pub,NaN,45.477268,9.174955,"[{'label': 'display', 'lat': 45.47726847977626...",1682,NaN,IT,NaN,NaN,Italia,[Italia],NaN,539b30e2498e62c5565878ba
9,Pubblico Soggiorno con Lovli,General Entertainment,"Piazza San Fedele, 1",45.466414,9.191126,"[{'label': 'display', 'lat': 45.46641388294476...",65,NaN,IT,Milano,Lombardia,Italia,"[Piazza San Fedele, 1, Milano Lombardia, Italia]",NaN,4fb6ed9c1081afad3ff529a6


In [85]:
search_query1 = 'bar'
radius = 3000

In [86]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=33ASWGTBDUB5DPU1Q35V3UGOFCESBNQOQPJ23QT2QV2RTYG3&client_secret=NIC4SFFHGVWIQ043S1BEKQRKAISNYODGGMDLJ4NSQ3THF103&ll=45.4668,9.1905&v=20180604&query=bar&radius=3000&limit=50'

In [87]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5fabfacad64a734725782bab'},
 'response': {'venues': [{'id': '4dac23bef7b145b370340045',
    'name': 'Bar Duomo',
    'location': {'address': 'Piazza del Duomo, 17',
     'lat': 45.464690110188315,
     'lng': 9.188503667820415,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.464690110188315,
       'lng': 9.188503667820415}],
     'distance': 281,
     'postalCode': '20123',
     'cc': 'IT',
     'neighborhood': 'Duomo',
     'city': 'Milano',
     'state': 'Lombardia',
     'country': 'Italia',
     'formattedAddress': ['Piazza del Duomo, 17',
      '20123 Milano Lombardia',
      'Italia']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1605106379',
    'hasPerk': False},
   {'id': '5228987d498e4429694

In [88]:
venues1 = results1['response']['venues']


dataframe1 = json_normalize(venues1)
dataframe1.head()

/opt/ibm/conda/miniconda/lib/python/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4dac23bef7b145b370340045,Bar Duomo,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1605106379,False,"Piazza del Duomo, 17",45.464690,9.188504,"[{'label': 'display', 'lat': 45.46469011018831...",281,20123,IT,Duomo,Milano,Lombardia,Italia,"[Piazza del Duomo, 17, 20123 Milano Lombardia,...",NaN,NaN
1,5228987d498e4429694882f8,Motta Caffè Bar Milano 1928,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1605106379,False,Piazza Duomo,45.464657,9.190182,"[{'label': 'display', 'lat': 45.46465740916136...",239,20121,IT,Duomo,Milano,Lombardia,Italia,"[Piazza Duomo, 20121 Milano Lombardia, Italia]",NaN,NaN
2,4bb892c47421a59316e1c140,Bar Straf,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",v-1605106379,False,Via San Raffaele 3,45.465153,9.191408,"[{'label': 'display', 'lat': 45.46515319542265...",196,20121,IT,Duomo,Milano,Lombardia,Italia,"[Via San Raffaele 3, 20121 Milano Lombardia, I...",38452989,NaN
3,4c3a2741ae2da593bfc503c6,Bar Brera,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",v-1605106379,False,"Via Brera, 23",45.472260,9.187459,"[{'label': 'display', 'lat': 45.47226020082689...",652,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Brera, 23 (via Fiori Chiari), 20121 Milan...",NaN,via Fiori Chiari
4,55391094498e7eb81261f2bc,Mandarin Bar & Bistrot,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",v-1605106379,False,Via Andegari 9,45.469719,9.191188,"[{'label': 'display', 'lat': 45.4697185364076,...",329,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Andegari 9, 20121 Milano Lombardia, Italia]",NaN,NaN


In [89]:
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Bar Duomo,Café,"Piazza del Duomo, 17",45.464690,9.188504,"[{'label': 'display', 'lat': 45.46469011018831...",281,20123,IT,Duomo,Milano,Lombardia,Italia,"[Piazza del Duomo, 17, 20123 Milano Lombardia,...",NaN,4dac23bef7b145b370340045
1,Motta Caffè Bar Milano 1928,Café,Piazza Duomo,45.464657,9.190182,"[{'label': 'display', 'lat': 45.46465740916136...",239,20121,IT,Duomo,Milano,Lombardia,Italia,"[Piazza Duomo, 20121 Milano Lombardia, Italia]",NaN,5228987d498e4429694882f8
2,Bar Straf,Cocktail Bar,Via San Raffaele 3,45.465153,9.191408,"[{'label': 'display', 'lat': 45.46515319542265...",196,20121,IT,Duomo,Milano,Lombardia,Italia,"[Via San Raffaele 3, 20121 Milano Lombardia, I...",NaN,4bb892c47421a59316e1c140
3,Bar Brera,Cocktail Bar,"Via Brera, 23",45.472260,9.187459,"[{'label': 'display', 'lat': 45.47226020082689...",652,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Brera, 23 (via Fiori Chiari), 20121 Milan...",via Fiori Chiari,4c3a2741ae2da593bfc503c6
4,Mandarin Bar & Bistrot,Hotel Bar,Via Andegari 9,45.469719,9.191188,"[{'label': 'display', 'lat': 45.4697185364076,...",329,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Andegari 9, 20121 Milano Lombardia, Italia]",NaN,55391094498e7eb81261f2bc
5,G. B. Bar,Bar,"Via Agnello,18",45.466096,9.192177,"[{'label': 'display', 'lat': 45.4660956216435,...",152,NaN,IT,NaN,Milano,Lombardia,Italia,"[Via Agnello,18, Milano Lombardia, Italia]",NaN,4c24a547c11dc9b644bb2524
6,Bar Chéri,Bar,NaN,45.465483,9.189272,"[{'label': 'display', 'lat': 45.46548299778277...",175,NaN,IT,NaN,NaN,NaN,Italia,[Italia],NaN,4d6bd2c071d3721ece539606
7,Bar Meda,Café,Piazza Filippo Meda,45.467001,9.193567,"[{'label': 'display', 'lat': 45.46700055277522...",240,20121,IT,NaN,Milano,Lombardia,Italia,"[Piazza Filippo Meda, 20121 Milano Lombardia, ...",NaN,4bd7d9af09ecb7137bbc477c
8,Si Bar,Italian Restaurant,NaN,45.465205,9.190164,"[{'label': 'display', 'lat': 45.4652045087736,...",179,NaN,IT,NaN,NaN,NaN,Italia,[Italia],NaN,527a65fe498ee2e254ffc33e
9,Coppola Bar Gelateria,Ice Cream Shop,NaN,45.464097,9.189310,"[{'label': 'display', 'lat': 45.46409725221743...",314,NaN,IT,Duomo,Milano,Lombardia,Italia,"[Milano Lombardia, Italia]",NaN,4c08e1ba7e3fc928e3f1f082


In [90]:
search_query2 = 'theatre, cinema'
radius = 3000

In [91]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=33ASWGTBDUB5DPU1Q35V3UGOFCESBNQOQPJ23QT2QV2RTYG3&client_secret=NIC4SFFHGVWIQ043S1BEKQRKAISNYODGGMDLJ4NSQ3THF103&ll=45.4668,9.1905&v=20180604&query=theatre, cinema&radius=3000&limit=50'

In [92]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5fabfaf1aa0cc20f98c11493'},
 'response': {'venues': [{'id': '4b058880f964a52036ca22e3',
    'name': 'The Space Cinema "Milano Odeon"',
    'location': {'address': 'Via Santa Radegonda, 8',
     'lat': 45.46490897785458,
     'lng': 9.192335307598114,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.46490897785458,
       'lng': 9.192335307598114}],
     'distance': 254,
     'postalCode': '20121',
     'cc': 'IT',
     'city': 'Milano',
     'state': 'Lombardia',
     'country': 'Italia',
     'formattedAddress': ['Via Santa Radegonda, 8',
      '20121 Milano Lombardia',
      'Italia']},
    'categories': [{'id': '4bf58dd8d48988d180941735',
      'name': 'Multiplex',
      'pluralName': 'Multiplexes',
      'shortName': 'Cineplex',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1605106417',
    'hasPerk': False},

In [93]:
venues2 = results2['response']['venues']


dataframe2 = json_normalize(venues2)
dataframe2.head()

/opt/ibm/conda/miniconda/lib/python/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id,location.crossStreet
0,4b058880f964a52036ca22e3,"The Space Cinema ""Milano Odeon""","[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",v-1605106417,False,"Via Santa Radegonda, 8",45.464909,9.192335,"[{'label': 'display', 'lat': 45.46490897785458...",254,20121,IT,Milano,Lombardia,Italia,"[Via Santa Radegonda, 8, 20121 Milano Lombardi...",NaN,NaN,NaN
1,5251971011d257e5dbfd61bd,Fendi Cinema,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1605106417,False,NaN,45.471351,9.194836,"[{'label': 'display', 'lat': 45.47135112901888...",609,NaN,IT,NaN,NaN,Italia,[Italia],NaN,NaN,NaN
2,4b058880f964a52033ca22e3,Cinema Mediolanum,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",v-1605106417,False,24 Corso Vittorio Emanuele,45.468059,9.190253,"[{'label': 'display', 'lat': 45.46805858325486...",141,20122,IT,Milano,Lombardia,Italia,"[24 Corso Vittorio Emanuele, 20122 Milano Lomb...",NaN,NaN,NaN
3,4b058880f964a5202aca22e3,Cinema Centrale Multisala,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",v-1605106417,False,Via Torino 30/32,45.462047,9.185953,"[{'label': 'display', 'lat': 45.4620465685442,...",637,20123,IT,Milano,Lombardia,Italia,"[Via Torino 30/32, 20123 Milano Lombardia, Ita...",NaN,NaN,NaN
4,4b058880f964a52032ca22e3,Cinema Manzoni,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",v-1605106417,False,"Via Alessandro Manzoni, 40",45.471291,9.194083,"[{'label': 'display', 'lat': 45.47129149888901...",572,20121,IT,Milano,Lombardia,Italia,"[Via Alessandro Manzoni, 40, 20121 Milano Lomb...",NaN,NaN,NaN


In [94]:
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns1]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,"The Space Cinema ""Milano Odeon""",Multiplex,"Via Santa Radegonda, 8",45.464909,9.192335,"[{'label': 'display', 'lat': 45.46490897785458...",254,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Santa Radegonda, 8, 20121 Milano Lombardi...",NaN,4b058880f964a52036ca22e3
1,Fendi Cinema,Art Gallery,NaN,45.471351,9.194836,"[{'label': 'display', 'lat': 45.47135112901888...",609,NaN,IT,NaN,NaN,NaN,Italia,[Italia],NaN,5251971011d257e5dbfd61bd
2,Cinema Mediolanum,Movie Theater,24 Corso Vittorio Emanuele,45.468059,9.190253,"[{'label': 'display', 'lat': 45.46805858325486...",141,20122,IT,NaN,Milano,Lombardia,Italia,"[24 Corso Vittorio Emanuele, 20122 Milano Lomb...",NaN,4b058880f964a52033ca22e3
3,Cinema Centrale Multisala,Movie Theater,Via Torino 30/32,45.462047,9.185953,"[{'label': 'display', 'lat': 45.4620465685442,...",637,20123,IT,NaN,Milano,Lombardia,Italia,"[Via Torino 30/32, 20123 Milano Lombardia, Ita...",NaN,4b058880f964a5202aca22e3
4,Cinema Manzoni,Movie Theater,"Via Alessandro Manzoni, 40",45.471291,9.194083,"[{'label': 'display', 'lat': 45.47129149888901...",572,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Alessandro Manzoni, 40, 20121 Milano Lomb...",NaN,4b058880f964a52032ca22e3
5,Cinema Hermes,Movie Theater,"Via Sant' Andrea, 21",45.469321,9.197347,"[{'label': 'display', 'lat': 45.46932079208096...",603,20121,IT,NaN,Milano,Lombardia,Italia,"[Via Sant' Andrea, 21, 20121 Milano Lombardia,...",NaN,4d270f5fd86aa090f0671dc0
6,Anteo Palazzo del Cinema,Multiplex,"Via Milazzo, 9",45.479727,9.187776,"[{'label': 'display', 'lat': 45.47972708428351...",1454,20121,IT,Brera,Milano,Lombardia,Italia,"[Via Milazzo, 9, 20121 Milano Lombardia, Italia]",NaN,4b058880f964a52022ca22e3
7,Cinema Cavour,Movie Theater,3 Piazza Cavour,45.472779,9.195071,"[{'label': 'display', 'lat': 45.4727790264127,...",755,20121,IT,NaN,Milano,Lombardia,Italia,"[3 Piazza Cavour, 20121 Milano Lombardia, Italia]",NaN,4b058880f964a52029ca22e3
8,Cinema Eliseo,Multiplex,"Via Torino, 64",45.460757,9.182706,"[{'label': 'display', 'lat': 45.46075698288565...",907,20123,IT,NaN,Milano,Lombardia,Italia,"[Via Torino, 64, 20123 Milano Lombardia, Italia]",NaN,4b058880f964a5202eca22e3
9,Cinema Auditorium San Carlo Pandora,Movie Theater,14 Corso Matteotti,45.467088,9.196019,"[{'label': 'display', 'lat': 45.46708785926344...",432,20122,IT,NaN,Milano,Lombardia,Italia,"[14 Corso Matteotti, 20122 Milano Lombardia, I...",NaN,4b058881f964a52057ca22e3


In [95]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)    
    
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)  
    
# display map
venues_map